In [1]:
%matplotlib inline

from keras.models import Sequential
from keras.layers import *
from keras.preprocessing.image import ImageDataGenerator

from keras.utils import np_utils
from scipy import io
import numpy as np
from sklearn.cross_validation import train_test_split
import string
import cv2
import os
import sys

import seaborn as sns
sns.set_style("white")

Using TensorFlow backend.
/home/ert/.conda/envs/tensorflow/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/ert/.conda/envs/tensorflow/lib/python3.5/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
train_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
        '/home/ert/Pobrane/rips/data_train',
        target_size=(150, 150),
        class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
        '/home/ert/Pobrane/rips/data_valid',
        target_size=(150, 150),
        class_mode='categorical')

Found 1560 images belonging to 2 classes.
Found 320 images belonging to 2 classes.


In [3]:
import progressbar

def load_folder(folder):
    bar = progressbar.ProgressBar()
    images = []
    i = 0
    for file in bar(os.listdir(folder)):
        image = cv2.imread(os.path.join(folder, file))
        if(image is not None):
            image = cv2.resize(image, (resolution_x, resolution_y), interpolation=cv2.INTER_LINEAR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            images.append(image)
        #if (i%10==0):
            #print (".", end="")
        #i+=1
    print ("")
    return np.array(images)

In [4]:
fobia_folder = "/home/ert/Pobrane/rips/data_train/reddit_sub_trypophobia"
normal_folder = "/home/ert/Pobrane/rips/data_train/reddit_sub_pics"
fobia_folder_val = "/home/ert/Pobrane/rips/data_valid/reddit_sub_trypophobia"
normal_folder_val = "/home/ert/Pobrane/rips/data_valid/reddit_sub_pics"
resolution_x = 150
resolution_y = 150
fobia_images = load_folder(fobia_folder) /255.
normal_images = load_folder(normal_folder) /255.
fobia_images_val = load_folder(fobia_folder_val) /255.
normal_images_val = load_folder(normal_folder_val) /255.

100% (712 of 712) |#################################################################################################################| Elapsed Time: 0:00:59 Time: 0:00:59


100% (848 of 848) |#################################################################################################################| Elapsed Time: 0:01:23 Time: 0:01:23


100% (150 of 150) |#################################################################################################################| Elapsed Time: 0:00:12 Time: 0:00:12
  0% (1 of 170) |                                                                                                                    | Elapsed Time: 0:00:00 ETA: 0:00:25

100% (170 of 170) |#################################################################################################################| Elapsed Time: 0:00:15 Time: 0:00:15


In [5]:
def image_to_X_Y(fobia_images, normal_images):
    X_train = [] 
    Y_train = [] # 1,0 -> fobia | 0,1 -> normal
    for img in fobia_images:
        X_train.append(img)
        Y_train.append([1,0])
    for img in normal_images:
        X_train.append(img)
        Y_train.append([0,1])
    X_train = np.array(X_train)
    Y_train = np.array(Y_train)
    return X_train, Y_train

X_train, Y_train = image_to_X_Y(fobia_images, normal_images)
X_test, Y_test = image_to_X_Y(fobia_images_val, normal_images_val)

In [6]:
model = Sequential()
model.add(Convolution2D(8, 3, 3, border_mode='same',
          input_shape=(150, 150, 3)))
#model.add(Convolution2D(8, 3, 3, border_mode='same'))
#model.add(Convolution2D(16, 3, 3, border_mode='same'))
#model.add(Activation('relu'))
model.add(Convolution2D(32, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(64, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(4, 4)))
model.add(Convolution2D(128, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(128, 3, 3, border_mode='same'))
model.add(Activation('relu'))
#model.add(Dropout(0.25))
model.add(GlobalAveragePooling2D())
#model.add(Flatten())
#for i in range(1):
#    model.add(Dense(128))
#    model.add(Activation('relu'))
#    #model.add(Dropout(0.1))
#for i in range(1):
#    model.add(Dense(32))
#    model.add(Activation('relu'))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [7]:
#model.fit_generator(
#        train_generator,
#        samples_per_epoch=1536,
#        nb_epoch=20,
#        validation_data=test_generator,
#        nb_val_samples=320)

In [ ]:
history = model.fit(X_train, Y_train,
                    nb_epoch=30,
                    batch_size=32,
                    validation_data=(X_test, Y_test))

Train on 1560 samples, validate on 320 samples
Epoch 1/30
1560/1560 [==============================] - 40s - loss: 0.4504 - acc: 0.7949 - val_loss: 0.5113 - val_acc: 0.7594

In [9]:
from keras_sequential_ascii import sequential_model_to_ascii_printout
sequential_model_to_ascii_printout(model)

      OPERATION           DATA DIMENSIONS   WEIGHTS(N)   WEIGHTS(%)

          Input   #####   (150, 150, 3)
  Convolution2D    \|/  -------------------       224     0.1%
                  #####   (150, 150, 8)
  Convolution2D    \|/  -------------------      2336     0.7%
           relu   #####   (150, 150, 32)
   MaxPooling2D   YYYYY -------------------         0     0.0%
                  #####   (75, 75, 32)
  Convolution2D    \|/  -------------------     18496     5.8%
           relu   #####   (75, 75, 64)
  Convolution2D    \|/  -------------------     36928    11.7%
           relu   #####   (75, 75, 64)
  Convolution2D    \|/  -------------------     36928    11.7%
           relu   #####   (75, 75, 64)
   MaxPooling2D   YYYYY -------------------         0     0.0%
                  #####   (18, 18, 64)
  Convolution2D    \|/  -------------------     73856    23.3%
           relu   #####   (18, 18, 128)
  Convolution2D    \|/  -------------------    147584    46.6%
        